# CPE342 - Karena Task1 V5

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.metrics import fbeta_score, make_scorer, precision_recall_curve
import joblib
import warnings
warnings.filterwarnings('ignore')

## Load Data

In [2]:
train_df = pd.read_csv("Dataset/task1/train.csv")
test_df = pd.read_csv("Dataset/task1/test.csv")

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")

Train shape: (99872, 34)
Test shape: (25889, 33)


## Remove Missing Target

In [3]:
target = 'is_cheater'
if train_df[target].isnull().any():
    print(f"Removing {train_df[target].isnull().sum()} rows with missing target")
    train_df = train_df.dropna(subset=[target])

print(f"\nClass distribution:")
print(train_df[target].value_counts())
print(train_df[target].value_counts(normalize=True))

Removing 2124 rows with missing target

Class distribution:
is_cheater
0.0    63619
1.0    34129
Name: count, dtype: int64
is_cheater
0.0    0.650847
1.0    0.349153
Name: proportion, dtype: float64


## Advanced Feature Engineering

In [4]:
def create_smart_features(df):
    """Create meaningful features"""
    df = df.copy()
    
    # --- PREPROCESSING: Outlier Handling ---
    # Clip reaction time to be at least 1ms
    if 'reaction_time_ms' in df.columns:
        df['reaction_time_ms'] = df['reaction_time_ms'].clip(lower=1)
        
    # 1. Suspicious skill indicators
    df['kd_headshot_product'] = df['kill_death_ratio'] * df['headshot_percentage']
    
    # 2. New account with high performance (red flag!)
    df['performance_per_day'] = (
        df['kill_death_ratio'] + df['win_rate'] + df['headshot_percentage']
    ) / (df['account_age_days'] + 1)
    
    # 3. Social isolation
    df['report_friend_ratio'] = df['reports_received'] / (df['friend_network_size'] + 1)
    
    # 4. Superhuman accuracy with fast reaction
    df['accuracy_reaction_ratio'] = df['accuracy_score'] / (df['reaction_time_ms'] + 1)
    
    # 5. Unnatural consistency
    df['consistency_kd_product'] = df['kill_consistency'] * df['kill_death_ratio']
    
    # 6. Combat dominance
    df['combat_score'] = (
        df['first_blood_rate'] * df['clutch_success_rate'] * df['damage_per_round']
    )
    
    # --- NEW FEATURES ---
    # 7. Skill Consistency (User Suggestion)
    df['skill_consistency'] = df['game_sense_score'] / (df['accuracy_score'] + 0.001)
    
    # 8. Impact Score
    df['impact_score'] = df['kill_death_ratio'] * df['win_rate']
    
    # 9. Precision Index
    df['precision_index'] = df['headshot_percentage'] * df['accuracy_score']
    
    # 10. Utility Efficiency
    df['utility_efficiency'] = df['utility_usage_rate'] * df['damage_per_round']
    
    # 11. Suspicion Score (Composite)
    df['suspicion_score'] = (df['kill_death_ratio'] * df['headshot_percentage']) / (df['account_age_days'] + 1)
    
    # --- V4 IMPROVEMENTS ---
    # 12. Aim-Movement Mismatch
    # Cheaters often have perfect aim but poor movement
    df['aim_movement_mismatch'] = df['aiming_smoothness'] / (df['movement_pattern_score'] + 0.01)
    
    # 13. Report Intensity
    # Reports normalized by playtime and account age
    df['report_intensity'] = df['reports_received'] / (df['avg_session_length_min'] * df['sessions_per_day'] * df['account_age_days'] + 1)
    
    # 14. Rapid Progression
    df['rapid_progression'] = df['level'] / (df['account_age_days'] + 1)
    
    # 15. Hardware Volatility
    df['hardware_volatility'] = df['device_changes_count'] / (df['account_age_days'] + 1)
    
    return df

print("\n✨ Creating strategic features (V4 Enhanced)...")
train_enhanced = create_smart_features(train_df)
test_enhanced = create_smart_features(test_df)
print(f"Enhanced train shape: {train_enhanced.shape}")


✨ Creating strategic features (V4 Enhanced)...
Enhanced train shape: (97748, 49)


In [5]:
# Cell 5: Prepare Features
features = [col for col in train_enhanced.columns 
           if col not in ['id', 'player_id', 'is_cheater']]

X_train = train_enhanced[features]
y_train = train_enhanced[target]
X_test = test_enhanced[features]

In [6]:
# Cell 6: ✨ IMPROVEMENT 2 - Better Imputation Strategy
# ใช้ mean สำหรับ features ที่มี missing น้อย, median สำหรับที่มีมาก
def smart_imputation(X_train, X_test):
    """Impute ตาม distribution ของแต่ละ feature"""
    
    X_train_imputed = X_train.copy()
    X_test_imputed = X_test.copy()
    
    for col in X_train.columns:
        missing_rate = X_train[col].isnull().sum() / len(X_train)
        
        if missing_rate > 0:
            # ถ้า missing มาก (>10%) ใช้ median ที่เสถียรกว่า
            if missing_rate > 0.1:
                imputer = SimpleImputer(strategy='median')
            else:
                # ถ้า missing น้อย ใช้ mean ที่รักษาค่าเฉลี่ย
                imputer = SimpleImputer(strategy='mean')
            
            X_train_imputed[col] = imputer.fit_transform(X_train[[col]])
            X_test_imputed[col] = imputer.transform(X_test[[col]])
    
    return X_train_imputed, X_test_imputed

print("\n✨ Smart imputation...")
X_train_imputed, X_test_imputed = smart_imputation(X_train, X_test)
print(f"Missing values after imputation: {X_train_imputed.isnull().sum().sum()}")


✨ Smart imputation...
Missing values after imputation: 0


In [7]:
# Cell 7: Calculate Class Weight
scale_pos_weight = (y_train == 0).sum() / y_train.sum()
print(f"\n⚖️ Scale pos weight: {scale_pos_weight:.2f}")


⚖️ Scale pos weight: 1.86


In [8]:
# Cell 8: ✨ IMPROVEMENT 3 - Optimized Hyperparameters
# ขยาย search space และเพิ่ม regularization
best_params = {
    'n_estimators': 500,  # เพิ่มจาก 100-400
    'learning_rate': 0.02,  # ลดลงเพื่อเรียนรู้ละเอียดขึ้น
    'num_leaves': 40,  # เพิ่มจาก 31-50
    'max_depth': 8,  # จำกัดความลึกป้องกัน overfit
    'min_child_samples': 25,  # เพิ่ม regularization
    'subsample': 0.8,  # bagging
    'colsample_bytree': 0.8,  # feature sampling
    'reg_alpha': 0.1,  # L1 regularization
    'reg_lambda': 0.1,  # L2 regularization
    'scale_pos_weight': scale_pos_weight,
    'random_state': 42,
    'verbose': -1
}

print("\n✨ Optimized hyperparameters:")
for key, value in best_params.items():
    if key not in ['random_state', 'verbose']:
        print(f"  {key}: {value}")


✨ Optimized hyperparameters:
  n_estimators: 500
  learning_rate: 0.02
  num_leaves: 40
  max_depth: 8
  min_child_samples: 25
  subsample: 0.8
  colsample_bytree: 0.8
  reg_alpha: 0.1
  reg_lambda: 0.1
  scale_pos_weight: 1.8640745407131765


In [9]:
# Cell 8.5: ✨ IMPROVEMENT 6 - Feature Selection
def select_features(X, y):
    """Select important features using LightGBM"""
    print("\n🔍 Performing Feature Selection...")
    
    # Train a quick LGBM to get feature importance
    lgbm = lgb.LGBMClassifier(n_estimators=500, random_state=42, verbose=-1)
    lgbm.fit(X, y)
    
    importances = pd.DataFrame({
        'feature': X.columns,
        'importance': lgbm.feature_importances_
    }).sort_values('importance', ascending=False)
    
    # Keep features with importance > 0 (or top N)
    # Here we drop features with 0 importance or very low relative importance
    zero_imp_features = importances[importances['importance'] == 0]['feature'].tolist()
    print(f"Dropping {len(zero_imp_features)} features with 0 importance: {zero_imp_features}")
    
    selected_feats = importances[importances['importance'] > 0]['feature'].tolist()
    return selected_feats

selected_features = select_features(X_train_imputed, y_train)
X_train_selected = X_train_imputed[selected_features]
X_test_selected = X_test_imputed[selected_features]
print(f"Selected {len(selected_features)} features.")


🔍 Performing Feature Selection...
Dropping 0 features with 0 importance: []
Selected 46 features.


In [10]:
# Cell 9: ✨ IMPROVEMENT 4 - Weighted Ensemble with SMOTE & Threshold Tuning
from scipy.optimize import minimize

def train_weighted_ensemble(X, y, n_splits=5):
    """Train Ensemble (LGBM, XGB, CatBoost) and find optimal weights"""
    
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    models = []
    cv_scores = []
    best_thresholds = []
    fold_weights = []
    
    # Base Models (Optimized)
    lgbm_params = {
        'n_estimators': 1000,
        'learning_rate': 0.02,
        'num_leaves': 40,
        'max_depth': 8,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'random_state': 42,
        'verbose': -1
    }
    
    xgb_params = {
        'n_estimators': 1000,
        'learning_rate': 0.02,
        'max_depth': 8,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'random_state': 42,
        'n_jobs': -1,
        'enable_categorical': True
    }
    
    cat_params = {
        'iterations': 1000,
        'learning_rate': 0.02,
        'depth': 8,
        'l2_leaf_reg': 3,
        'random_seed': 42,
        'verbose': 0,
        'allow_writing_files': False
    }
    
    print(f"Starting {n_splits}-Fold Cross-Validation with Weighted Ensemble...")
    
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y), 1):
        X_fold_train, X_fold_val = X.iloc[train_idx], X.iloc[val_idx]
        y_fold_train, y_fold_val = y.iloc[train_idx], y.iloc[val_idx]
        
        # 1. Apply SMOTE only on Training Data
        smote = SMOTE(random_state=42)
        X_resampled, y_resampled = smote.fit_resample(X_fold_train, y_fold_train)
        
        # 2. Train Models
        lgbm = lgb.LGBMClassifier(**lgbm_params)
        lgbm.fit(X_resampled, y_resampled, eval_set=[(X_fold_val, y_fold_val)], 
                 callbacks=[lgb.early_stopping(50, verbose=False)])
        
        xgb_model = xgb.XGBClassifier(**xgb_params)
        xgb_model.fit(X_resampled, y_resampled, eval_set=[(X_fold_val, y_fold_val)], 
                      verbose=False)
        
        cat_model = CatBoostClassifier(**cat_params)
        cat_model.fit(X_resampled, y_resampled, eval_set=[(X_fold_val, y_fold_val)], 
                      early_stopping_rounds=50)
        
        # 3. Get Predictions
        p1 = lgbm.predict_proba(X_fold_val)[:, 1]
        p2 = xgb_model.predict_proba(X_fold_val)[:, 1]
        p3 = cat_model.predict_proba(X_fold_val)[:, 1]
        
        # 4. Optimize Weights
        def f2_loss(weights):
            w1, w2, w3 = weights
            # Normalize
            total = w1 + w2 + w3
            if total == 0: return 1
            w1, w2, w3 = w1/total, w2/total, w3/total
            
            avg_prob = w1*p1 + w2*p2 + w3*p3
            
            # Find best threshold for these weights
            precisions, recalls, thresholds = precision_recall_curve(y_fold_val, avg_prob)
            f2_scores = (5 * precisions * recalls) / (4 * precisions + recalls + 1e-10)
            return -np.max(f2_scores) # Minimize negative F2
            
        res = minimize(f2_loss, [1, 1, 1], bounds=[(0,1), (0,1), (0,1)], method='SLSQP')
        best_w = res.x / np.sum(res.x)
        
        # 5. Calculate Score with Best Weights
        avg_prob = best_w[0]*p1 + best_w[1]*p2 + best_w[2]*p3
        precisions, recalls, thresholds = precision_recall_curve(y_fold_val, avg_prob)
        f2_scores = (5 * precisions * recalls) / (4 * precisions + recalls + 1e-10)
        best_idx = np.argmax(f2_scores)
        best_threshold = thresholds[best_idx]
        best_score = f2_scores[best_idx]
        
        print(f"Fold {fold}: Best F2 = {best_score:.4f} (Weights: L={best_w[0]:.2f}, X={best_w[1]:.2f}, C={best_w[2]:.2f})")
        
        models.append((lgbm, xgb_model, cat_model))
        cv_scores.append(best_score)
        best_thresholds.append(best_threshold)
        fold_weights.append(best_w)
    
    avg_threshold = np.mean(best_thresholds)
    avg_weights = np.mean(fold_weights, axis=0)
    print(f"\n📊 Mean F2-score: {np.mean(cv_scores):.4f}")
    print(f"⚖️  Average Weights: LGBM={avg_weights[0]:.2f}, XGB={avg_weights[1]:.2f}, Cat={avg_weights[2]:.2f}")
    print(f"🎯 Average Optimal Threshold: {avg_threshold:.4f}")
    
    return models, avg_threshold, avg_weights

print("\n🚀 Training Weighted Ensemble with SMOTE...")
models, best_threshold, best_weights = train_weighted_ensemble(
    X_train_selected, 
    y_train
)


🚀 Training Weighted Ensemble with SMOTE...
Starting 5-Fold Cross-Validation with Weighted Ensemble...
Fold 1: Best F2 = 0.8367 (Weights: L=0.33, X=0.33, C=0.33)
Fold 2: Best F2 = 0.8368 (Weights: L=0.33, X=0.33, C=0.33)
Fold 3: Best F2 = 0.8365 (Weights: L=0.33, X=0.33, C=0.33)
Fold 4: Best F2 = 0.8336 (Weights: L=0.33, X=0.33, C=0.33)
Fold 5: Best F2 = 0.8396 (Weights: L=0.33, X=0.33, C=0.33)

📊 Mean F2-score: 0.8366
⚖️  Average Weights: LGBM=0.33, XGB=0.33, Cat=0.33
🎯 Average Optimal Threshold: 0.1827


In [11]:
# Cell 10: ✨ IMPROVEMENT 5 - Weighted Ensemble Prediction
def ensemble_predict_final(models, X, weights, threshold=0.5):
    """Predict using all CV models and weighted average"""
    final_probs = np.zeros(len(X))
    
    for (lgbm, xgb_model, cat_model) in models:
        p1 = lgbm.predict_proba(X)[:, 1]
        p2 = xgb_model.predict_proba(X)[:, 1]
        p3 = cat_model.predict_proba(X)[:, 1]
        
        # Weighted Average for this fold
        fold_prob = weights[0]*p1 + weights[1]*p2 + weights[2]*p3
        final_probs += fold_prob
    
    # Average across all folds
    final_probs /= len(models)
    
    # Apply optimized threshold
    return (final_probs >= threshold).astype(int)

print("\n🔮 Making weighted ensemble predictions...")
test_predictions = ensemble_predict_final(models, X_test_selected, best_weights, threshold=best_threshold)

# Feature Importance (Average from LGBM models)
print("\n📊 Feature Importance (Average LGBM):")
importances = np.zeros(len(X_train_selected.columns))
for (lgbm, _, _) in models:
    importances += lgbm.feature_importances_
importances /= len(models)

feature_imp = pd.DataFrame({
    'feature': X_train_selected.columns,
    'importance': importances
}).sort_values('importance', ascending=False)
print(feature_imp.head(15))

# Save Model
print("\n💾 Saving model...")
model_filename = 'Model/anti_cheat_weighted_V5'
joblib.dump({
    'models': models,
    'feature_names': list(X_train_selected.columns),
    'best_threshold': best_threshold,
    'best_weights': best_weights
}, model_filename)
print(f"Model saved to {model_filename}")


🔮 Making weighted ensemble predictions...

📊 Feature Importance (Average LGBM):
                    feature  importance
4          reports_received      3754.2
0       friend_network_size      3577.2
1          account_age_days      3394.2
2       crosshair_placement      1577.8
5          game_sense_score      1410.2
24      report_friend_ratio      1332.0
6                     level      1320.4
45     device_changes_count      1096.8
33      kd_headshot_product      1080.2
3   level_progression_speed      1003.8
31          precision_index       826.8
18      headshot_percentage       721.0
7       spray_control_score       716.4
8         aiming_smoothness       713.0
41          suspicion_score       638.2

💾 Saving model...
Model saved to Model/anti_cheat_weighted_V5


In [12]:
test_predictions

array([1, 0, 1, ..., 1, 0, 0], shape=(25889,))

## Make submission

In [13]:
submission_df = pd.read_csv("final_submission.csv")
target_column = 'task1'

In [15]:
print(f"Filling '{target_column}' column with predictions...")
submission_df[target_column] = test_predictions
submission_df.to_csv("final_submission.csv", index=False)
print(submission_df.head(20))

Filling 'task1' column with predictions...
          id  task1  task2          task3  task4  task5
0   ANS00001      1      2     517.156944      1      0
1   ANS00002      0      0    1413.611996      0      0
2   ANS00003      1      0  158344.753397      3      1
3   ANS00004      0      0     117.963622      4      0
4   ANS00005      1      0     101.542132      4      0
5   ANS00006      1      2     609.682928      3      0
6   ANS00007      1      1       0.000000      1      0
7   ANS00008      1      0   17357.013216      0      0
8   ANS00009      1      0       0.000000      3      0
9   ANS00010      0      1       0.000000      0      0
10  ANS00011      1      2     128.685354      0      0
11  ANS00012      0      0     432.884720      3      0
12  ANS00013      0      0      25.834546      2      0
13  ANS00014      0      1    1670.033716      0      0
14  ANS00015      1      2       0.000000      4      0
15  ANS00016      0      0       0.000000      2      0
16  A